# Riset Komunikasi 4.0: Pengolahan dan Analisis Data Menggunakan Python 
### oleh: Sigit Purnomo

Notebook ini merupakan materi workshop pada Forum Studi Komunikasi, Program Studi Ilmu Komunikasi, Fakultas Ilmu Sosial dan Ilmu Politik, Universitas Atma Jaya Yogyakarta

## Pengumpulan Data Media Sosial Twitter dengan Python
Bagian notebook ini digunakan untuk mengumpulkan data dari media sosial Twitter berdasarkan kata kunci atau hashtags tertentu. Kebutuhan yang harus dipenuhi adalah:
1. Python
2. Tweepy
3. Token Twitter

### Langkah 1: Inisialisasi library yang dibutuhkan
Langkah pertama yang harus dilakukan adalah mengimport library-library yang akan digunakan. Library utama yang digunakan dalam kasus ini adalah Tweepy yang berfungsi untuk mengakses Twitter API.

In [17]:
import tweepy

import csv
import pandas as pd

import re
from re import finditer

import ftfy
from string import punctuation
import emoji

from pathlib import Path

### Langkah 2: Inisialisasi akses Twitter API
Langkah kedua yang harus dilakukan adalah melakukan koneksi ke Twitter API dengan Tweepy. Untuk melakukan koneksi ke Twitter API, Bapak/Ibu sudah harus memiliki Twitter Credentials yang meliputi:
    1. Consumer Key
    2. Consumer Secret
    3. Access Token
    4. Access Token Secret    NB: Jangan gunakan key yang sudah ada dibawah
    

In [1]:
consumer_key = #insert your consumer key here
consumer_secret = #insert your consumer key here
access_token = #insert your consumer key here
access_token_secret = #insert your consumer key here
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

SyntaxError: invalid syntax (<ipython-input-1-afe96d28f3ae>, line 1)

### Langkah 3: Inisialisasi File
Langkah ketiga yang harus dilakukan adalah melakukan insialisasi untuk membuat file yang akan kita gunakan untuk menyimpan data tweet yang akan dikumpulkan dengan menggunakan library CSV. Struktur file CSV dapat disesuaikan dengan kebutuhan. Untuk kasus ini, strukturnya adalah sebagai berikut:
1. Tweet Date
2. Tweet ID
3. Tweet Source
4. Tweet User
5. Tweer User Location
6. Tweet User Follower Count
7. Tweet User Friend Count
8. Tweet User Status Count
9. Tweet Hashtags
10. Tweet User Mentions
11. Tweet Symbols
12. Tweet URLS
13. Tweet Emoticons
14. Tweet Emoji
15. Tweet Text
   

In [20]:
# Open/Create a file to append data
keyword_hashtag = '2019gantipresiden'
data_folder = Path("C:\\Users\\Kevin Antariksa")
file_data = keyword_hashtag + '.csv'
filename = data_folder / file_data
csvFile = open(filename, 'a',encoding = "utf-8")
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['tweet_date', 'tweet_id', 'tweet_source', 
                    'tweet_user', 'tweet_user_location',
                    'tweet_user_follower', 'tweet_user_friends',
                    'tweet_user_nstatuses','tweet_hashtags',
                    'tweet_mentions', 'tweet_text'])

166

### Langkah 4: Pembuatan Fungsi untuk Membersihkan Data Tweet
Langkah keempat yang harus dilakukan adalah membuat fungsi untuk membersihkan data teks dari Tweet. Fungsi ini bertujuan untuk mengambil teks saja dan membuang hal-hal berikut:
1. Hashtags
2. User Mentions
3. Symbols
4. URLs
5. Emoticons
6. Emoji
    

In [21]:
def fix_text(text):
    text = text.replace('\n', '')
    text = ' '.join(text.split(' '))
    return text

def extract_text(data):
    text = data.full_text
    slices = []
    # Strip out the user mentions.
    if data.entities.get('user_mentions'):
        for men in data.entities.get('user_mentions'):
            slices += [{'start': men['indices'][0], 'stop': men['indices'][1]}]
    # Strip out the hashtags.
    if data.entities.get('hashtags'):
        for tag in data.entities.get('hashtags'):
            slices += [{'start': tag['indices'][0], 'stop': tag['indices'][1]}]
    # Strip out the urls
    if data.entities.get('urls'):
        for urlshort in data.entities.get('urls'):
            slices += [{'start': urlshort['indices'][0], 'stop': urlshort['indices'][1]}]
    # Strip out the media.
    if data.entities.get('media'):
        for med in data.entities.get('media'):
            slices += [{'start': med['indices'][0], 'stop': med['indices'][1]}]
    # Sort the slices from highest start to lowest.
    slices = sorted(slices, key=lambda x: -x['start'])
    # No offsets, since we're sorted from highest to lowest.
    for s in slices:
        text = text[:s['start']] + text[s['stop']:]
    # Remove punctuation
    text = fix_text(text)
    for p in punctuation:
        text = text.replace(p,'')

### Langkah 5: Pembuatan Fungsi untuk Mengambil Data Hashtags, User Mentions, URLs, dan Symbols dari Tweet
Langkah kelima yang harus dilakukan adalah membuat fungsi untuk mengambil data hashtags, user mentions, urls, dan symbols dari data Tweet    

In [22]:
# Gets all hashtags.
def get_hashtags(data):
    if data.entities.get('hashtags'):
        return [tag['text'] for tag in data.entities.get('hashtags')]

# Gets all symbols.
def get_symbols(data):
    if data.entities.get('symbols'):
        return [sym['text'] for sym in data.entities.get('symbols')]

# Gets all urls.
def get_urls(data):
    if data.entities.get('urls'):
        return [url['url'] for url in data.entities.get('urls')]

# Gets the screen names of any user mentions.
def get_user_mentions(data):
    if data.entities.get('user_mentions'):
        return [m['screen_name'] for m in data.entities.get('user_mentions')]


### Langkah 6: Pembuatan Fungsi untuk Mengambil Data Emoticons dan Emoji dari Tweet
Langkah kelima yang harus dilakukan adalah membuat fungsi untuk mengambil data emoticons dan emoji dari data Tweet    

In [23]:
def get_emoticons(data):
    emoticon = ''
    # Happy Emoticons
    emoticons_happy = set([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
    ])
    # Sad Emoticons
    emoticons_sad = set([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
    ])
    # all emoticons (happy + sad)
    emoticons = emoticons_happy.union(emoticons_sad)
    for emo in emoticons:
        str_emo = data
        if emo in str_emo.lower():
            emoticon = emoticon + ' ' + emo
    return emoticon
def get_emoji(data):
    emojis = ''
    str_emoji = emoji.demojize(data)

    for match in finditer(':\w*_*\w:', str_emoji):
        emojis = emojis + ' ' + match.group()
    return emojis

### Langkah 7: Mengambil Data Tweet berdasar Hashtags/Kata Kunci Tertentu
Langkah ketujuh yang harus dilakukan adalah memanggil fungsi Tweepy untuk mengambil data Tweet berdasar hashtags/kata kunci tertentu dengan memanfaatkan Cursor. Data tweet yang diambil kemudian disimpan ke file yang telah dibuat di awal.
#### Catatan:
Data tweet yang bisa ditarik dibatasi untuk 7 hari kebelakang.

In [24]:
for tweet in tweepy.Cursor(api.search,q=keyword_hashtag + " -RT",count=1000,
                           lang="id", tweet_mode='extended',
                           since="2019-01-01").items():
    _hashtags = ''
    _symbols = ''
    _mentions = ''
    _urls = ''
    _created = tweet.created_at
    _id = tweet.id_str
    _source = re.sub('<[^<]+?>', '', tweet.source)
    _user = tweet.user.screen_name
    _location = tweet.user.location
    _nfollowers = str(tweet.user.followers_count)
    _nfriends = str(tweet.user.friends_count)
    _nstatuses = str(tweet.user.statuses_count)
    if tweet.entities.get('hashtags'): 
        _hashtags = ' '.join(get_hashtags(tweet))
    if tweet.entities.get('user_mentions'):     
        _mentions = ' '.join(get_user_mentions(tweet))
    if tweet.entities.get('symbols'): 
        _symbols = ' '.join(get_symbols(tweet))
    if tweet.entities.get('urls'): 
        _urls = ' '.join(get_urls(tweet))
    _emoticons = ' '.join(get_emoticons(tweet.full_text))
    _emoji = ' '.join(get_emoji(tweet.full_text))
    _text = extract_text(tweet)
    #print (tweet.created_at, tweet.text)
    csvWriter.writerow([_created, _id, _source, 
                        _user, _location, _nfollowers,
                        _nfriends, _nstatuses, _hashtags, 
                        _mentions,  _text])

### Langkah 8: Menampilkan Data
Langkah kedelapan yang harus dilakukan adalah menampilkan data yang telah dikumpulkan untuk diolah lebih lanjut.

In [25]:
print(filename)

C:\Users\Kevin Antariksa\DoaBersamaTragedi2122Mei.csv


In [26]:
dataset = pd.read_csv(filename,encoding = "ISO-8859-1")
dataset.shape

(6955, 11)

In [27]:
dataset['tweet_user'].describe()

count           6955
unique          1928
top       bayubakrie
freq             105
Name: tweet_user, dtype: object

In [28]:
dataset['tweet_source'].value_counts()

Twitter for Android        5270
Twitter Web App             903
Twitter Web Client          329
Twitter for iPhone          267
IFTTT                        56
TweetDeck                    37
Mobile Web (M2)              30
Twidere for Android #7       22
TweetCaster for Android       9
Silver Bird                   7
Tween                         5
Bad Kitty                     4
Echofon                       4
Twitter for iPad              3
Krile2                        3
Instagram                     2
Mixero                        1
Twitter Media Studio          1
TweetCaster                   1
FoxRT                         1
Name: tweet_source, dtype: int64

In [30]:
dataset.head(10)

,tweet_date,tweet_id,tweet_source,tweet_user,tweet_user_location,tweet_user_follower,tweet_user_friends,tweet_user_nstatuses,tweet_hashtags,tweet_mentions,tweet_text
0,2019-06-01 10:18:05,1134766055824347136,Twitter for Android,cacihmalaka,NaN,638,1292,2022,DoaBersamaTragedi2122Mei DoaBersamaTragedi2122Mei,Niputujelantik sukasukaskip Dio_Nzee,setuju bebaskan indonesia dari jeratan cina ko...
1,2019-06-01 10:14:08,1134765060553752576,Twitter for Android,uyab_mihaf,NaN,15,26,1374,DoaBersamaTragedi2122Mei,NaN,kita adalah makhluk langit kenapa biarkan terk...
2,2019-06-01 10:12:36,1134764677794226177,Twitter for Android,Ssnart1,"DKI Jakarta, Indonesia",604,1018,1470,DoaBersamaTragedi2122Mei,ummuRDzA22 Abengridwan81 ordinarywomen_ amir47...,siap folbek
3,2019-06-01 10:10:32,1134764157910151168,Twitter for Android,rendyrhmn,pontianak,402,891,1840,DoaBersamaTragedi2122Mei,RRabrusun prabowo sandiuno,waalaikum salam wrwb
4,2019-06-01 10:09:42,1134763947049881600,Twitter for Android,Hendrian_Rahman,"Bandung, Indonesia",499,701,236,DoaBersamaTragedi2122Mei,NaN,bpk kok gitu ditanya dalang kerusuhan 22 mei b...
5,2019-06-01 09:54:51,1134760209153290240,Twitter for Android,udin_nsp1,NaN,322,1755,6789,InternationalCriminalCourtSOS DoaBersamaTraged...,aburasyid13,ada apa pada negeri ku fitnah2 caki maki sumpa...
6,2019-06-01 09:47:43,1134758413013929990,Twitter for Android,ArisSet18676370,NaN,43,183,321,DoaBersamaTragedi2122Mei IbuAniHusnulKhatimah,prabowo aziz_ua,turut berduka semoga ibu ani khusnul khatimaha...
7,2019-06-01 09:46:24,1134758081340985344,Twitter for Android,farmproxy,NaN,488,736,804,DoaBersamaTragedi2122Mei DoaBersamaTragedi2122Mei,NaN,wah maaf teteh baru follow
8,2019-06-01 09:39:45,1134756410883883009,Twitter for Android,tahrizy,"Agam, Sumatera Barat",145,320,2524,DoaBersamaTragedi2122Mei,MichelAdamNew,prestasi lbp g berani terjun payung
9,2019-06-01 09:35:44,1134755398492123136,Twitter for Android,Sifa77599186,NaN,9,23,219,DoaBersamaTragedi2122Mei DoaBersamaTragedi2122Mei,rizqi_property msaid_didu,kita menyatu semakin kuat


In [13]:
dataset.tail(10)

,tweet_date,tweet_id,tweet_source,tweet_user,tweet_user_location,tweet_user_follower,tweet_user_friends,tweet_user_nstatuses,tweet_hashtags,tweet_mentions,tweet_text,label,sasaran_hatespeech,subjek_hatespeech;
9306,2019-05-22 13:12:08,1131185981170827264,IFTTT,PinondangSbrng,NaN,885,342,9356,NaN,NaN,personel tni kompak shalawat dengan massa aksi,NaN,NaN,NaN
9307,2019-05-22 13:12:08,1131185980118130690,IFTTT,Kerja4As,NaN,3342,271,5830,NaN,NaN,personel tni kompak shalawat dengan massa aksi,NaN,NaN,NaN
9308,2019-05-22 13:02:00,1131183431067418624,Twitter for Android,linaanggriani07,"cilacap,jawa tengah",253,181,41035,2019GantiPresiden,NaN,terimalah itu kekalahan dr orang2 yg gak perna...,NaN,NaN,NaN
9309,2019-05-22 12:57:17,1131182240354963456,IFTTT,PinondangSbrng,NaN,885,342,9356,NaN,NaN,surat terbuka rizal ramli korban sudah terlalu...,NaN,NaN,NaN
9310,2019-05-22 12:57:16,1131182238157217793,IFTTT,PinondangSbrng,NaN,885,342,9356,NaN,NaN,ini pernyataan lengkap prabowo soal rusuh 22 mei,NaN,NaN,NaN
9311,2019-05-22 12:57:10,1131182212530016256,IFTTT,Kerja4As,NaN,3342,271,5830,NaN,NaN,massa ricuh di slipi bentangkan poster habib r...,NaN,NaN,NaN
9312,2019-05-22 12:57:09,1131182209354948609,IFTTT,Kerja4As,NaN,3342,271,5830,NaN,NaN,surat terbuka rizal ramli korban sudah terlalu...,NaN,NaN,NaN
9313,2019-05-22 12:22:16,1131173430441775105,IFTTT,PinondangSbrng,NaN,885,342,9356,NaN,NaN,fadli zon ungkap perbuatan keji oknum kepolisi...,NaN,NaN,NaN
9314,2019-05-22 12:22:16,1131173428852133888,IFTTT,PinondangSbrng,NaN,885,342,9356,NaN,NaN,kiai nu sejatim kegaduhan ini akumulasi dari p...,NaN,NaN,NaN
9315,2019-05-22 12:22:15,1131173424150339585,IFTTT,Kerja4As,NaN,3342,271,5830,NaN,NaN,jelang maghrib kericuhan pecah kembali di slipi,NaN,NaN,NaN


In [32]:
dataset['tweet_text']

0       setuju bebaskan indonesia dari jeratan cina ko...
1       kita adalah makhluk langit kenapa biarkan terk...
2                                             siap folbek
3                                    waalaikum salam wrwb
4       bpk kok gitu ditanya dalang kerusuhan 22 mei b...
5       ada apa pada negeri ku fitnah2 caki maki sumpa...
6       turut berduka semoga ibu ani khusnul khatimaha...
7                             wah maaf teteh baru follow 
8                     prestasi lbp g berani terjun payung
9                              kita menyatu semakin kuat 
10                                                 folbek
11      semoga mereka yang berbuat kejam mendaptkan ba...
12                             seisi negri menjadi dungu 
13      ya alloh di saksikan langit tujuh dan bumi tuj...
14      di jaman rezim kodok tol semakin membuat masya...
15                              udah follback duluan dek 
16                  pake ngayal biar bisa loncat loncat  
17      begitu

In [33]:
dataset[dataset['tweet_user_location'] == 'Indonesia']

,tweet_date,tweet_id,tweet_source,tweet_user,tweet_user_location,tweet_user_follower,tweet_user_friends,tweet_user_nstatuses,tweet_hashtags,tweet_mentions,tweet_text
10,2019-06-01 09:33:56,1134754945557680128,Twitter for Android,WanAbdulKarimL1,Indonesia,1300,1359,358,DoaBersamaTragedi2122Mei,RedWhite1945,folbek
12,2019-06-01 09:09:56,1134748906951069696,Twitter for Android,acretta,Indonesia,175,692,1486,PurnawirawanTNIBersamaRakyat PrabowoJanganTemu...,NaN,seisi negri menjadi dungu
27,2019-06-01 08:13:43,1134734757554675712,Twitter Web Client,HaluanKita,Indonesia,1605,1520,5969,PrabowoIroni DoaBersamaTragedi2122Mei,NaN,prabowo yang kalah kenapa jokowi yang harus me...
49,2019-06-01 06:54:03,1134714710782164992,Twitter for Android,Dwijayani2,Indonesia,40,210,692,InternationalCriminalCourtSOS DoaBersamaTraged...,Jay_F618,br kali ini ada mentri gak punya solusi masih ...
57,2019-06-01 06:41:14,1134711486528364546,Twitter for Android,acretta,Indonesia,175,692,1486,InternationalCriminalCourtSOS PrabowoJanganTem...,NaN,nasionalisme menuju kehancuran
61,2019-06-01 06:38:04,1134710689128337408,Twitter Web Client,notbuzzer,Indonesia,85739,48796,147368,PrabowoKalahJokowiDimintaMengalah DoaBersamaTr...,NaN,cara buzzer putarbalikan fakta dengan hoax dan...
63,2019-06-01 06:36:39,1134710329911283712,Twitter Web Client,HaluanKita,Indonesia,1605,1520,5969,PrabowoKalahJokowiDimintaMengalah DoaBersamaTr...,NaN,mereka mati kenapa yang disalahkan jokowi suda...
64,2019-06-01 06:36:38,1134710328430694400,Twitter Web Client,HaluanKita,Indonesia,1605,1520,5969,PrabowoKalahJokowiDimintaMengalah DoaBersamaTr...,NaN,karena penyeldikan butuh waktu diwaktu yg seng...
65,2019-06-01 06:36:38,1134710327000543232,Twitter Web Client,HaluanKita,Indonesia,1605,1520,5969,PrabowoKalahJokowiDimintaMengalah DoaBersamaTr...,NaN,karena tau persis aksi 2122 mei ulahnya siapa ...
66,2019-06-01 06:36:36,1134710320503508992,Twitter Web Client,HaluanKita,Indonesia,1605,1520,5969,PrabowoKalahJokowiDimintaMengalah DoaBersamaTr...,NaN,kubu oposisi lewat para buzzernya dimedsos sed...


### Setelah dilakukan pengambilan data, data tsb akan diolah
Pertama: diolah menggunakan TF IDF

In [34]:
##yang mau ditanyakan, satu tagar apakah sudah cukup, mau dipisah seperti apa-> menjadi 0/1(hatespeech/buukan)